In [1]:
!pip install keras_applications
!pip install tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import tensorflow as tf
import pandas as pd 
#import tensorflow_addons as tfa
from tensorflow_datasets import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
#from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
#from tensorflow.keras.applications.xception import Xception, preprocess_input 
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.experimental import CosineDecayRestarts
from tensorflow.keras.regularizers import l1, l2, l1_l2
from keras import backend as K

# places 
#from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.layer_utils import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils

import time
from matplotlib import pyplot as plt, rc
import os
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import shutil
import re
from glob import glob

from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from sklearn.cluster import DBSCAN
from collections import Counter

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

from PIL import Image
import csv
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import euclidean_distances
from tensorflow.keras import layers


from sklearn.preprocessing import MinMaxScaler as MMS

import cv2
from PIL import Image
from scipy import stats

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
#from tensorflow.keras.applications.xception import Xception, preprocess_input

# places365에 필요한 
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.layer_utils import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils
import matplotlib.image as img

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
!nvidia-smi

Sat Apr  9 07:54:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   48C    P0    38W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

In [4]:
cd '/home/ubuntu/image_used_crowdSourcing/'

/home/ubuntu/image_used_crowdSourcing


<style>
    span { color: #aba; }
</style>
## <span style="color:#cba"> 이미지 가져오기
---

In [5]:
df=pd.read_csv('/home/ubuntu/pair_df.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df.drop_duplicates(inplace=True)

In [17]:
score_df = pd.DataFrame(data = {'image':list(df['left_image'])+list(df['right_image'])})

In [6]:
image_size= (448, 448)
image_size_channel = (224, 224, 3)
AUTOTUNE = tf.data.AUTOTUNE

def prepro_image(filepath):
    image = tf.io.read_file(filepath) # CPU나 GPU 사용할 때
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size) 
    return image

In [7]:
left_path = tf.data.Dataset.from_tensor_slices(list(df['left_image']))
left = left_path.map(prepro_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

right_path = tf.data.Dataset.from_tensor_slices(list(df['right_image']))
right = right_path.map(prepro_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

binary_label = tf.data.Dataset.from_tensor_slices(list(df['label']))


cat = []
for i in list(df['label']):
  if i==0:
    cat.append(np.array([1,0]))
  else:
    cat.append(np.array([0,1]))

cat_label=tf.data.Dataset.from_tensor_slices(np.array(cat))

pair_images=tf.data.Dataset.zip((left, right))
cat_binary_labels = tf.data.Dataset.zip((cat_label, cat_label, binary_label))
dataset = tf.data.Dataset.zip((pair_images, cat_binary_labels))

# validation
ratio = 0.8
train_val_dataset = dataset.take(round(len(df) * ratio))
train_dataset = train_val_dataset.take(round(len(df) * ratio * ratio))
val_dataset = train_val_dataset.skip(round(len(df) * ratio * ratio))
test_images = dataset.skip(round(len(df) * ratio))
test_dataset = dataset.skip(round(len(df) * ratio)) 


# shuffle
batch_size = 32 
drop_remainder = False

train_dataset = train_dataset.shuffle(buffer_size=2000, reshuffle_each_iteration=True)
val_dataset = val_dataset.shuffle(buffer_size=2000, reshuffle_each_iteration=True)

train_dataset = train_dataset.batch(batch_size, drop_remainder=drop_remainder)
train_dataset = train_dataset.prefetch(batch_size) 

val_dataset = val_dataset.batch(batch_size, drop_remainder=drop_remainder)
val_dataset = val_dataset.prefetch(batch_size)

test_images = test_images.batch(batch_size, drop_remainder=drop_remainder)
test_images = test_images.prefetch(32)


<style>
    span { color: #aba; }
</style>
## <span style="color:#cba"> VGG_Places365
---

In [8]:
def VGG16_Places365(include_top=True, weights='places',
                    input_tensor=None, input_shape=None,
                    pooling=None,
                    classes=365):
    """Instantiates the VGG16-places365 architecture.
    Optionally loads weights pre-trained
    on Places. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
                 'places' (pre-training on Places),
                 or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`, or invalid input shape
        
        """
    WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'
    WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'

    if not (weights in {'places', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `places` '
                         '(pre-training on Places), '
                         'or the path to the weights file to be loaded.')

    if weights == 'places' and include_top and classes != 365:
        raise ValueError('If using `weights` as places with `include_top`'
                         ' as true, `classes` should be 365')


    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten =include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # Block 1
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv1')(img_input)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dropout(0.5, name='drop_fc1')(x)

        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dropout(0.5, name='drop_fc2')(x)
        
        x = Dense(365, activation='softmax', name="predictions")(x)

    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    model = Model(inputs, x, name='vgg16-places365')

    # load weights
    if weights == 'places':
        if include_top:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')

        model.load_weights(weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')

    elif weights is not None:
        model.load_weights(weights)

    return model


<style>
    span { color: #aba; }
</style>
## <span style="color:#cba"> RSS-CNN architecture
---

In [9]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

def global_extraction(): # using ResNet50 network without FC layer

    i = Input(shape=(224, 224, 3))
    vgg_model = VGG16_Places365(weights = 'places',
                                    include_top = False,
                                    input_tensor=i)
    
    model = tf.keras.Sequential() 

    for layer in vgg_model.layers[:-1]:
        model.add(layer)
    
    model.add(GlobalAveragePooling2D())

    for layer in model.layers[:-3]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
  
    return model

def feature_extraction_network(): # using ResNet50 network without FC layer. 여기 고침

    i = Input(shape=(224, 224, 3))
    vgg_model = VGG16_Places365(weights = 'places',
                                    include_top = False,
                                    input_tensor=i)
    
    model = tf.keras.Sequential()

    for layer in vgg_model.layers[:-1]: 
        model.add(layer)
    
    model.add(GlobalAveragePooling2D())

    for layer in model.layers[:-3]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
  
    return model

def build_model(input_dim, feature_extraction_network, global_extraction):
        
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
      tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)])
    
    input_left = Input(shape=input_dim, name='input_left')
    input_right = Input(shape=input_dim, name='input_right')

    # augmentation --> preprocess_input 
    aug_left = data_augmentation(input_left) # output shape (none, 448, 448, 3)
    aug_right = data_augmentation(input_right)

    # image resize - global에 대해서만
    global_left = Lambda(lambda x: tf.image.resize(x, (224, 224)))(aug_left)
    global_right = Lambda(lambda x: tf.image.resize(x, (224, 224)))(aug_right)   

    # imge preprocess 
    
    pp_glb_left = preprocess_input(global_left)
    pp_glb_right = preprocess_input(global_right)

    # patch
    #feature_extraction_network = feature_extraction_network()

    left_feature1 = feature_extraction_network(preprocess_input(aug_left[:, 0:224, 0:224, :]))
    left_feature2 = feature_extraction_network(preprocess_input(aug_left[:, 0:224, 224:448, :]))
    left_feature3 = feature_extraction_network(preprocess_input(aug_left[:, 224:448, 0:224, :]))
    left_feature4 = feature_extraction_network(preprocess_input(aug_left[:, 224:448, 224:448, :]))

    right_feature1 = feature_extraction_network(preprocess_input(aug_right[:, 0:224, 0:224, :]))
    right_feature2 = feature_extraction_network(preprocess_input(aug_right[:, 0:224, 224:448, :]))
    right_feature3 = feature_extraction_network(preprocess_input(aug_right[:, 224:448, 0:224, :]))
    right_feature4 = feature_extraction_network(preprocess_input(aug_right[:, 224:448, 224:448, :]))

    left_concat = Concatenate(name='left_concat')([left_feature1, left_feature2, left_feature3, left_feature4])
    right_concat = Concatenate(name='right_concat')([right_feature1, right_feature2, right_feature3, right_feature4])

    concat = Concatenate(name='feature_concat')([left_concat, right_concat])
    x1 = Dropout(rate=0.5)(concat)
    x1 = Dense(512, activation='relu')(x1)
    x1 = Dropout(rate=0.5)(x1)
    patches_loss = Dense(2, activation='softmax', name='patches')(x1)


    # global
    #global_extraction = feature_extraction_network()
    left_global = global_extraction(pp_glb_left)
    right_global = global_extraction(pp_glb_right)

    global_concat = Concatenate()([left_global, right_global])
    x2 = Dropout(rate=0.5)(global_concat)
    x2 = Dense(256, activation='relu')(x2)
    x2 = Dropout(rate=0.5)(x2)
    glb_loss = Dense(2, activation='softmax', name='global')(x2)
    
    # score
    left_total_concat = Concatenate()([left_concat, left_global])
    right_total_concat = Concatenate()([right_concat, right_global])
    
    score_model = tf.keras.Sequential([Dropout(rate=0.5), Dense(256, activation='relu'), Dropout(rate=0.5), Dense(1)])
    
    left_score = score_model(left_total_concat)
    right_score = score_model(right_total_concat)

    diff = Subtract()([right_score, left_score])

    # Pass difference through sigmoid function.
    prob = Activation("sigmoid", name='ranking')(diff)

    # Build model.
    model = Model(inputs = [input_left, input_right], outputs = [patches_loss, glb_loss, prob])
    
    return model, score_model

<style>
    span { color: #aba; }
</style>
## <span style="color:#cba"> model training
---

In [8]:
# input_shape = (4, 28, 28, 3)
input_dim = (448, 448, 3)
#preprocess = preprocess()
feature_extraction_network = feature_extraction_network()
global_extraction = global_extraction()
model, score_model = build_model(input_dim, feature_extraction_network, global_extraction)

In [46]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left (InputLayer)        [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_right (InputLayer)       [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_34 (Sequential)     (None, 448, 448, 3)  0           ['input_left[0][0]',             
                                                                  'input_right[0][0]']      

In [10]:
make_folder = f'/home/ubuntu/train_0410_patch_aug_global_sep_again0409/'
if not os.path.exists(make_folder):
  os.makedirs(make_folder) 

In [13]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # with strategy.scope():
    input_dim = (448, 448, 3)
    feature_extraction_network = feature_extraction_network()
    global_extraction = global_extraction()
    model, score_model = build_model(input_dim, feature_extraction_network, global_extraction)

    model.compile(
    optimizer = tf.keras.optimizers.Adam(0.00001), 
    loss = ["categorical_crossentropy", "categorical_crossentropy", 'binary_crossentropy'],
    metrics='accuracy'
    )
    mcp_cb = ModelCheckpoint(filepath=f'{make_folder}/'+'weights.{epoch:02d}-{val_loss:.3f}.hdf5', monitor='val_loss',
                        save_best_only=False, save_weights_only=True, mode='min', period=1, verbose=1)
    #rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=5, patience=2, mode='min', verbose=1)
    # ely_cb = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)
    #model.fit(x=[np.array([left]), np.array([right])], y=[np.array([np.array([0, 1])]), np.array([1])], epochs=1, validation_split=0.0, batch_size=None)
    history = model.fit(train_dataset, 
          epochs=15, 
          validation_data=val_dataset,
          callbacks=[mcp_cb])
          # validation_split은 dataset은 사용하지 못한다

    #model.fit(x=[np.array([left]), np.array([right])], y=[np.array([0, 1]), np.array([1])], epochs=10, validation_split=0.0, batch_size=None)
    #model.fit(x=pair_batch, y=[cat_label, binary_label], epochs=10, validation_split=0.0, batch_size=None)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
58925056/58909656 [==============================] - 3s 0us/step
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/15
INFO:tensorflow:batch_all_reduce: 20 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:lo

In [11]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # with strategy.scope():
    input_dim = (448, 448, 3)
    feature_extraction_network = feature_extraction_network()
    global_extraction = global_extraction()
    model, score_model = build_model(input_dim, feature_extraction_network, global_extraction)
    
    model.load_weights('/home/ubuntu/weights.09-1.990.hdf5') 

    model.compile(
    optimizer = tf.keras.optimizers.Adam(0.00001), 
    loss = ["categorical_crossentropy", "categorical_crossentropy", 'binary_crossentropy'],
    metrics='accuracy'
    )
    mcp_cb = ModelCheckpoint(filepath=f'{make_folder}/'+'weights.{epoch:02d}-{val_loss:.3f}.hdf5', monitor='val_loss',
                        save_best_only=False, save_weights_only=True, mode='min', period=1, verbose=1)
    #rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=5, patience=2, mode='min', verbose=1)
    # ely_cb = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)
    #model.fit(x=[np.array([left]), np.array([right])], y=[np.array([np.array([0, 1])]), np.array([1])], epochs=1, validation_split=0.0, batch_size=None)
    history = model.fit(train_dataset, 
          epochs=5, 
          validation_data=val_dataset,
          callbacks=[mcp_cb])
          # validation_split은 dataset은 사용하지 못한다

    #model.fit(x=[np.array([left]), np.array([right])], y=[np.array([0, 1]), np.array([1])], epochs=10, validation_split=0.0, batch_size=None)
    #model.fit(x=pair_batch, y=[cat_label, binary_label], epochs=10, validation_split=0.0, batch_size=None)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/5
INFO:tensorflow:batch_all_reduce: 20 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:loc

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
# with strategy.scope():
  input_dim = image_size_channel
  feature_extraction_network = feature_extraction_network()
  model, score_model = build_model(input_dim, feature_extraction_network)
    
  model.load_weights('/home/ubuntu/train_0401_KIN/weights.10-1.314.hdf5') 

  model.compile(
    optimizer = tf.keras.optimizers.Adam(0.00005), 
    loss = ["categorical_crossentropy", 'binary_crossentropy'],
    metrics='accuracy'
    )
  mcp_cb = ModelCheckpoint(filepath=f'{make_folder}/'+'weights.{epoch:02d}-{val_loss:.3f}.hdf5', monitor='val_loss',
                        save_best_only=False, save_weights_only=True, mode='min', period=1, verbose=1)
  #rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=5, patience=2, mode='min', verbose=1)
  # ely_cb = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)
  #model.fit(x=[np.array([left]), np.array([right])], y=[np.array([np.array([0, 1])]), np.array([1])], epochs=1, validation_split=0.0, batch_size=None)
  history = model.fit(
          train_dataset, 
          epochs=7, 
          validation_data=val_dataset,
          callbacks=[mcp_cb])
          # validation_split은 dataset은 사용하지 못한다

  #model.fit(x=[np.array([left]), np.array([right])], y=[np.array([0, 1]), np.array([1])], epochs=10, validation_split=0.0, batch_size=None)
  #model.fit(x=pair_batch, y=[cat_label, binary_label], epochs=10, validation_split=0.0, batch_size=None)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/7
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:loc